In [2]:
import easyocr
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import numpy as np
from Levenshtein import distance as levenshtein_distance

/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-02 20:02:48.554110: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 20:02:49.113690: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-02 20:02:49.113715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered

In [ ]:
# Инициализация моделей
easyocr_reader = easyocr.Reader(['ru', 'en'])  # Укажите нужные языки
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
trocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

def evaluate_ocr(image_path, true_text):
    # EasyOCR
    easyocr_result = easyocr_reader.readtext(image_path, detail=0)
    easyocr_text = " ".join(easyocr_result)
    
    # TrOCR
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values
    generated_ids = trocr_model.generate(pixel_values)
    trocr_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # Метрики
    def word_accuracy(pred, true):
        pred_words = pred.split()
        true_words = true.split()
        correct = sum(1 for p, t in zip(pred_words, true_words) if p == t)
        return correct / max(len(true_words), 1)
    
    return {
        "easyocr_word_acc": word_accuracy(easyocr_text, true_text),
        "trocr_word_acc": word_accuracy(trocr_text, true_text),
        "easyocr_levenshtein": levenshtein_distance(easyocr_text, true_text),
        "trocr_levenshtein": levenshtein_distance(trocr_text, true_text),
    }

# Пример для одного изображения
metrics = evaluate_ocr("/home/lastinm/PROJECTS/credit_cards_detection/notebooks/output/2_1742460932042_resized_jpg.rf.cc580bf6003a8859a97b0c846b08415c_crop0.jpg",
                        "4276 0400 1647 7684")
print(metrics)

NameError: name 'easyocr' is not defined